In [1]:
import pandas as pd
from docx import Document
from docx2pdf import convert
import smtplib
import os.path
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [2]:
gmail_user = 'drhscomunicahidreletricas@gmail.com'
gmail_password = 'Sparta1941'

try:
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
except:
    print('Something went wrong...')

In [3]:
def send_email(subject, message, from_email, to_email=[], attachment=[]):
    """
    :param subject: email subject
    :param message: Body content of the email (string), can be HTML/CSS or plain text
    :param from_email: Email address from where the email is sent
    :param to_email: List of email recipients, example: ["a@a.com", "b@b.com"]
    :param attachment: List of attachments, exmaple: ["file1.txt", "file2.txt"]
    """
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = ", ".join(to_email)
    msg.attach(MIMEText(message, 'html'))

    for f in attachment:
        with open(f, 'rb') as a_file:
            basename = os.path.basename(f)
            part = MIMEApplication(a_file.read(), Name=basename)

        part['Content-Disposition'] = 'attachment; filename="%s"' % basename
        msg.attach(part)

    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    server.sendmail(from_email, to_email, msg.as_string())

In [4]:
df = pd.read_csv("DRHS Comunica - Monitoramento das Hidrelétricas - presença.csv")

In [5]:
df.head()

,Carimbo de data/hora,Nome completo:,E-mail:,Profissional:,Qual(is) usuário(s) de água representa? (separar por vírgulas),Qual(is) empreendimento(s) representa? (separar por vírgulas)
0,2021/04/06 10:54:19 AM GMT-3,Amanda Fadel,amanda-fadel@sema.rs.gov.br,Órgão Gestor,NaN,NaN
1,2021/04/06 10:59:17 AM GMT-3,Roberto Mentzingen Rolo,roberto-rolo@sema.rs.gov.br,Analista SIOUT,NaN,NaN


In [6]:
for idx, row in df.iterrows():
    
    doc=Document('modelo_certificado.docx')
    
    nome = row['Nome completo:']
    nome = nome.upper()
    email = row['E-mail:']
    
    for p in doc.paragraphs:
        if 'nome' in p.text:
            inline = p.runs
            # Loop added to work with runs (strings with same style)
            for i in range(len(inline)):
                if 'nome' in inline[i].text:
                    text = inline[i].text.replace('nome', nome)
                    inline[i].text = text
    
    doc.save('certificados/{}.docx'.format(nome))
    convert('certificados/{}.docx'.format(nome), 'certificados/{}.pdf'.format(nome))
    arquivo_pdf = 'certificados/{}.pdf'.format(nome)
    
    #send_email(subject='Certificado DRHS Comunica - Monitoramento das Hidrelétricas', message= 'Segue em anexo o certificado de participação do evento DRHS Comunica - Monitoramento das Hidrelétricas.', from_email=gmail_user, to_email=[email], attachment=[arquivo_pdf])

print('Done!')

Done!
